### Description:
Explore and clean lab variables

- First grab by lab_names (or base_names), then check their base_names (or lab_names), from all_labs file (labs that we went through to select)
- Then reverse to include lab_names with more than 1 base_names and base_names with more than 1 lab_names.
    - Based off Triage work of selecting lab_names, take those final lab_names and get their base_names
    - Add a few more base_names 
    - Use these base_names to run m6_labs.sql. This makes sure we have everything from Triage selection.
- Check the units and ranges of these labs to keep only valid ones.
- Combine lab_names/base_names and convert to same units as needed --> rename these groups
- Most urine related labs are all NA

It would be shorter to use base_name, but still need to check because some same labs have different base_names
- For example, new shc_core_2021 has lab_name `BUN, Arterial` and `BUN, Peripheral` which has base_name `BUN`. Lab_name wasn't available in old data
- Added 'BUN, Arterial' and 'BUN, Peripheral', 'BUN, Venous' under lab_name list (NEW in 2021)

- For ord_num_value = 9999999 of extreme cases on either end (hi/lo, panic hi/lo):
    - Remove those observations where no < > in ord_value: cancelled or N/A
    - Use ord_value to take bounds values. For example, if ord_value is <0.2 and ord_num_value is 9999999, then replace 9999999 with 0.2

Output file: 
- `6_6_coh3_labs`
- `6_6_cohort3_withlabs`: cohort information only from 2_4_coh3_labs

### Importing R libraries

In [1]:
library(bigrquery)  # to query STARR-OMOP (stored in BigQuery) using SQL
library(tidyverse)
library(lubridate)
# library(mice)
# library(VIM) # for missing data plot

# library(data.table)
# library(Matrix)
# library(caret) # import this before glmnet to avoid rlang version problem
# library(glmnet)
# library(bit64)

# library(slam)
# library(mtools) for one hot coder, not available on Nero or use caret or tidyr
options(repr.matrix.max.rows=250, repr.matrix.max.cols=30)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




### Set up and run queries
Do this on Nero GCP as querying from a local computer takes much longer time

In [2]:
# CREDENTIALS depending on LOCATIONS:
# credential <- paste0("/home/", "minh084", "/.config/gcloud/application_default_credentials.json")

# local computer
# credential <- "C:/Users/User/AppData/Roaming/gcloud/application_default_credentials.json"

# Nero onprem
# credential <- "/home/minh084/.config/gcloud/application_default_credentials.json"

# Nero gcp notebook
credential <- "/home/jupyter/.config/gcloud/application_default_credentials.json"

project_id <- "som-nero-phi-jonc101"

Sys.setenv(GOOGLE_APPLICATION_CREDENTIALS = credential)
Sys.setenv(GCLOUD_PROJECT = project_id)
gargle::credentials_app_default()

NULL

In [3]:
library(DBI)
con <- dbConnect(
  bigrquery::bigquery(),
  project = project_id,
  dataset = "shc_core_2021" #, billing = project_id
)
con 
dbListTables(con)

<BigQueryConnection>
  Dataset: som-nero-phi-jonc101.shc_core_2021
  Billing: som-nero-phi-jonc101

[1] "adt"                 "alert"               "alert_history"      
 [4] "alerts_orders"       "allergy"             "alt_com_action"     
 [7] "clinical_doc_meta"   "culture_sensitivity" "demographic"        
[10] "dep_map"             "diagnosis"           "drg_code"           
[13] "encounter"           "f_ip_hsp_admission"  "family_hx"          
[16] "flowsheet"           "lab_result"          "lda"                
[19] "med_orderset"        "myc_mesg"            "ndc_code"           
[22] "order_comment"       "order_med"           "order_proc"         
[25] "order_quest"         "pharmacy_mar"        "proc_note"          
[28] "proc_orderset"       "procedure"           "prov_map"           
[31] "social_hx"           "temp_table"          "treatment_team"     
[34] "yn_jon_chen_lpch"

### LABS
1. Use lab results as is. no manually process as with vital signs. 
2. Some notes about labs:
- Cannot trust "result_in_range_yn" --> remove this
- There is 1 incident of Glucose by Meter = hi in ord_value
- ord_value has text, blanks and #
- ord_num_value has NA, 9999999, #. The blanks from ord_value are NA in ord_num_value
- result_flag: "Abnormal" for text in ord_value, usually correct. However, when ord_value is N/A or < or >, the ord_num_value is 9999999.
3. Use ord_num_value and result_flag are good
4. All the urine labs start with X seem to be NA. Remove mag and phos as these are low count 1K

In [4]:
# read inputs
datadir6 = "../../DataTD/validation"
valdir = "../../OutputTD/6_validation"

cohort <- read.csv(file.path(valdir, "6_5_cohort3.csv"))
labs0 <- read.csv(file.path(datadir6, "labs_2021.csv")) # 1386585

# check for unique CSNs and MRNs
nrow(cohort) #44258
nrow(labs0) #depending on which cohort_demo used to query labs 1863768
nrow(labs0 %>% select(pat_enc_csn_id_coded) %>% distinct()) # 41724 vs 41627
colnames(labs0)

[1] 16704

[1] 514511

[1] 15982

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "inpatient_data_id_coded" "admit_time"             
 [5] "label"                   "DBP"                    
 [7] "Pulse"                   "RR"                     
 [9] "SBP"                     "Temp"                   
[11] "order_id_coded"          "lab_name"               
[13] "base_name"               "ord_value"              
[15] "ord_num_value"           "reference_low"          
[17] "reference_high"          "reference_unit"         
[19] "result_in_range_yn"      "result_flag"            
[21] "result_time_utc"         "order_time_utc"         
[23] "taken_time_utc"

In [5]:
labs0 <- labs0 %>% mutate(reference_unit = factor(reference_unit),
                          reference_low = factor(reference_low),
                          reference_high = factor(reference_high),
                          result_in_range_yn = factor(result_in_range_yn),
                          result_flag = factor(result_flag)) #%>% 

In [6]:
options(repr.matrix.max.rows=200, repr.matrix.max.cols=20)
# labs0 %>% group_by(base_name, lab_name) %>% count() %>% arrange(base_name)
labs0 %>% group_by(base_name, lab_name) %>% count() %>% arrange(lab_name)

base_name,lab_name,n
<chr>,<chr>,<int>
ALB,"Albumin, Ser/Plas",14756
ALKP,"Alk P'TASE, Total, Ser/Plas",14723
ALT,"ALT (SGPT), Ser/Plas",14724
AG,Anion Gap,15483
PCO2A,Arterial pCO2 for POC,1
PHA,Arterial pH for POC,1
PO2A,Arterial pO2 for POC,1
AST,"AST (SGOT), Ser/Plas",14724
BE,Base Excess (vt),333


In [7]:
labs0 %>% group_by(base_name) %>% count(sort=TRUE)

base_name,n
<chr>,<int>
GLU,18790
HGB,16798
HCT,16793
K,16548
NA,16506
CR,16289
EGFR,16286
PLT,15828
MCH,15788


In [8]:
# from Triage
triage_labs <- c("Platelet count", "Total Bilirubin, Ser/Plas", "Total Bilirubin", 
"TROPONIN I", "Troponin I, POCT", "WBC", "WBC count", 
"Hematocrit", "Hct, ISTAT", "Hct(Calc), ISTAT", 
"Hct (Est)", "HCT, POC", 
"Hematocrit (Manual Entry) See EMR for details", 
"Hemoglobin", "Hgb(Calc), ISTAT", "Hgb, calculated, POC", "HgB", 
"Potassium, Ser/Plas", "Potassium, ISTAT",  "Potassium, Whole Bld", 
"Potassium, whole blood, ePOC", "Sodium, Ser/Plas", "Sodium, ISTAT",  
"Sodium, Whole Blood", "Chloride, Ser/Plas",  "Chloride, ISTAT",  
"Chloride, Whole Bld", "Creatinine, Ser/Plas", "Creatinine,ISTAT", 
"Anion Gap", "Anion Gap, ISTAT", "Glucose, Ser/Plas", "Glucose,ISTAT", 
"Glucose, Whole Blood", "Glucose by Meter", "Glucose, Non-fasting", 
"BUN, Ser/Plas", "BUN, ISTAT", "Urea Nitrogen,Ser/Plas", 'BUN, Arterial', 'BUN, Peripheral', 'BUN, Venous',
"Neutrophil, Absolute", "NEUT, ABS", "Neut, ABS (Seg+Band) (man diff)", 
"Neutrophils, Absolute (Manual Diff)", "Neut, ABS (Seg+Band) (man diff)", 
"Basophil, Absolute", "BASOS, ABS", "Basophils, ABS (man diff)", 
"Baso, ABS (man diff)", "Eosinophil, Absolute", "EOS, ABS", 
"Eosinophils, ABS (man diff)", "Eos, ABS (man diff)", 
"Lymphocyte, Absolute", "LYM, ABS", "Lymphocytes, ABS (man diff)", 
"Lym, ABS (man diff)", "Lymphocytes, Abs.", "Monocyte, Absolute", 
"MONO, ABS", "Monocytes, ABS (man diff)", "Mono, ABS (man diff)", 
"Lactate, ISTAT", "Lactate, Whole Bld", "Lactic Acid", "HCO3", 
"HCO3 (a), ISTAT", "Bicarbonate, Art for POC", "HCO3 (v), ISTAT", 
"HCO3, ISTAT", "O2 Saturation (a)", "O2 Saturation, ISTAT", 
"Oxygen Saturation for POC", "O2 Saturation (v)",  
"O2 Saturation, ISTAT (Ven)", "pCO2 (a)", "pCO2 (a), ISTAT",  
"PCO2, ISTAT", "Arterial pCO2 for POC", "pCO2 (v)", "PCO2 (v), ISTAT", 
"pH (a)", "PH (a), ISTAT", "pH by Meter", "Arterial pH for POC", 
"pH (v)", "PH (v), ISTAT", "PH, ISTAT", "pO2 (a)", "PO2 (a), ISTAT",  
"PO2, ISTAT", "Arterial pO2 for POC", "pO2 (v)", "PO2 (v), ISTAT", 
"tCO2", "TCO2 (a), ISTAT","TCO2, ISTAT", "TCO2, (ISTAT)", 
"CO2 Arterial Total for POC", "INR", "INR, ISTAT", 
"Prothrombin Time", "PT, ISTAT")
str_sort(triage_labs)

[1] "Anion Gap"                                    
  [2] "Anion Gap, ISTAT"                             
  [3] "Arterial pCO2 for POC"                        
  [4] "Arterial pH for POC"                          
  [5] "Arterial pO2 for POC"                         
  [6] "Baso, ABS (man diff)"                         
  [7] "Basophil, Absolute"                           
  [8] "Basophils, ABS (man diff)"                    
  [9] "BASOS, ABS"                                   
 [10] "Bicarbonate, Art for POC"                     
 [11] "BUN, Arterial"                                
 [12] "BUN, ISTAT"                                   
 [13] "BUN, Peripheral"                              
 [14] "BUN, Ser/Plas"                                
 [15] "BUN, Venous"                                  
 [16] "Chloride, ISTAT"                              
 [17] "Chloride, Ser/Plas"                           
 [18] "Chloride, Whole Bld"                          
 [19] "CO2 Arterial Total for POC"                   
 [20] "Creatinine, Ser/Plas"                         
 [21] "Creatinine,ISTAT"                             
 [22] "EOS, ABS"                                     
 [23] "Eos, ABS (man diff)"                          
 [24] "Eosinophil, Absolute"                         
 [25] "Eosinophils, ABS (man diff)"                  
 [26] "Glucose by Meter"                             
 [27] "Glucose, Non-fasting"                         
 [28] "Glucose, Ser/Plas"                            
 [29] "Glucose, Whole Blood"                         
 [30] "Glucose,ISTAT"                                
 [31] "HCO3"                                         
 [32] "HCO3 (a), ISTAT"                              
 [33] "HCO3 (v), ISTAT"                              
 [34] "HCO3, ISTAT"                                  
 [35] "Hct (Est)"                                    
 [36] "Hct, ISTAT"                                   
 [37] "HCT, POC"                                     
 [38] "Hct(Calc), ISTAT"                             
 [39] "Hematocrit"                                   
 [40] "Hematocrit (Manual Entry) See EMR for details"
 [41] "Hemoglobin"                                   
 [42] "HgB"                                          
 [43] "Hgb, calculated, POC"                         
 [44] "Hgb(Calc), ISTAT"                             
 [45] "INR"                                          
 [46] "INR, ISTAT"                                   
 [47] "Lactate, ISTAT"                               
 [48] "Lactate, Whole Bld"                           
 [49] "Lactic Acid"                                  
 [50] "LYM, ABS"                                     
 [51] "Lym, ABS (man diff)"                          
 [52] "Lymphocyte, Absolute"                         
 [53] "Lymphocytes, ABS (man diff)"                  
 [54] "Lymphocytes, Abs."                            
 [55] "MONO, ABS"                                    
 [56] "Mono, ABS (man diff)"                         
 [57] "Monocyte, Absolute"                           
 [58] "Monocytes, ABS (man diff)"                    
 [59] "NEUT, ABS"                                    
 [60] "Neut, ABS (Seg+Band) (man diff)"              
 [61] "Neut, ABS (Seg+Band) (man diff)"              
 [62] "Neutrophil, Absolute"                         
 [63] "Neutrophils, Absolute (Manual Diff)"          
 [64] "O2 Saturation (a)"                            
 [65] "O2 Saturation (v)"                            
 [66] "O2 Saturation, ISTAT"                         
 [67] "O2 Saturation, ISTAT (Ven)"                   
 [68] "Oxygen Saturation for POC"                    
 [69] "pCO2 (a)"                                     
 [70] "pCO2 (a), ISTAT"                              
 [71] "pCO2 (v)"                                     
 [72] "PCO2 (v), ISTAT"                              
 [73] "PCO2, ISTAT"                                  
 [74] "pH (a)"                                       
 [75] 

In [9]:
# labs0[labs0$lab_name == "Platelets", ]$base_name
# labs0[labs0$lab_name %in% lab_names, ]$base_name   
# lapply(lab_names, function(x) labs0[labs0$lab_name == x, ]$basename)

In [10]:
# check overlapping of Triage labs and the new queried labs
# str_sort(unique(labs0$base_name))
# str_sort(intersect(labs0$lab_name, lab_names))

setdiff(triage_labs, labs0$lab_name) # NONE
str_sort(setdiff(labs0$lab_name, triage_labs)) # in A, not B

triage_labs0_names <- labs0 %>% filter(lab_name %in% triage_labs) %>% distinct(lab_name, base_name)
triage_labs0_base_names <- str_sort(unique(triage_labs0_names$base_name))
triage_labs0_lab_names <- str_sort(unique(triage_labs0_names$lab_name))

triage_labs0_base_names; # triage_labs0_lab_names

str_sort(setdiff(labs0$base_name, triage_labs0_base_names))

[1] "Total Bilirubin"                              
 [2] "WBC count"                                    
 [3] "Hct(Calc), ISTAT"                             
 [4] "Hematocrit (Manual Entry) See EMR for details"
 [5] "Chloride, ISTAT"                              
 [6] "Anion Gap, ISTAT"                             
 [7] "BUN, ISTAT"                                   
 [8] "Urea Nitrogen,Ser/Plas"                       
 [9] "NEUT, ABS"                                    
[10] "Neutrophils, Absolute (Manual Diff)"          
[11] "BASOS, ABS"                                   
[12] "Baso, ABS (man diff)"                         
[13] "EOS, ABS"                                     
[14] "Eos, ABS (man diff)"                          
[15] "LYM, ABS"                                     
[16] "Lym, ABS (man diff)"                          
[17] "Lymphocytes, Abs."                            
[18] "MONO, ABS"                                    
[19] "Mono, ABS (man diff)"                         
[20] "TCO2, (ISTAT)"

[1] "Albumin, Ser/Plas"            "Alk P'TASE, Total, Ser/Plas" 
 [3] "ALT (SGPT), Ser/Plas"         "AST (SGOT), Ser/Plas"        
 [5] "Base Excess (vt)"             "Base Excess Arterial for POC"
 [7] "Base Excess, ISTAT"           "Calcium, Ser/Plas"           
 [9] "CO2, Ser/Plas"                "eGFR"                        
[11] "Globulin"                     "MCH"                         
[13] "Protein, Total, Ser/Plas"     "RDW"                         
[15] "Sodium, whole blood, ePOC"

[1] "AG"      "BASOAB"  "BUN"     "CL"      "CR"      "EOSAB"   "GLU"    
 [8] "HCO3"    "HCO3A"   "HCO3V"   "HCT"     "HGB"     "INR"     "K"      
[15] "LAC"     "LACWBL"  "LYMAB"   "MONOAB"  "NEUTAB"  "NEUTABS" "O2SATA" 
[22] "O2SATV"  "PCO2A"   "PCO2V"   "PH"      "PHA"     "PHV"     "PLT"    
[29] "PO2A"    "PO2V"    "PT"      "TBIL"    "TCO2A"   "TNI"     "WBC"    
[36] NA

[1] "ALB"  "ALKP" "ALT"  "AST"  "BE"   "CA"   "CO2"  "EGFR" "GLOB" "MCH" 
[11] "RDW"  "TP"

In [11]:
# get lab_names and base_names from labs0
labs0_names <- labs0 %>% distinct(lab_name, base_name)
base_names <- str_sort(unique(labs0_names$base_name))
lab_names <- str_sort(unique(labs0_names$lab_name))

lab_names

[1] "Albumin, Ser/Plas"               "Alk P'TASE, Total, Ser/Plas"    
 [3] "ALT (SGPT), Ser/Plas"            "Anion Gap"                      
 [5] "Arterial pCO2 for POC"           "Arterial pH for POC"            
 [7] "Arterial pO2 for POC"            "AST (SGOT), Ser/Plas"           
 [9] "Base Excess (vt)"                "Base Excess Arterial for POC"   
[11] "Base Excess, ISTAT"              "Basophil, Absolute"             
[13] "Basophils, ABS (man diff)"       "Bicarbonate, Art for POC"       
[15] "BUN, Arterial"                   "BUN, Peripheral"                
[17] "BUN, Ser/Plas"                   "BUN, Venous"                    
[19] "Calcium, Ser/Plas"               "Chloride, Ser/Plas"             
[21] "Chloride, Whole Bld"             "CO2 Arterial Total for POC"     
[23] "CO2, Ser/Plas"                   "Creatinine, Ser/Plas"           
[25] "Creatinine,ISTAT"                "eGFR"                           
[27] "Eosinophil, Absolute"            "Eosinophils, ABS (man diff)"    
[29] "Globulin"                        "Glucose by Meter"               
[31] "Glucose, Non-fasting"            "Glucose, Ser/Plas"              
[33] "Glucose, Whole Blood"            "Glucose,ISTAT"                  
[35] "HCO3"                            "HCO3 (a), ISTAT"                
[37] "HCO3 (v), ISTAT"                 "HCO3, ISTAT"                    
[39] "Hct (Est)"                       "Hct, ISTAT"                     
[41] "HCT, POC"                        "Hematocrit"                     
[43] "Hemoglobin"                      "HgB"                            
[45] "Hgb, calculated, POC"            "Hgb(Calc), ISTAT"               
[47] "INR"                             "INR, ISTAT"                     
[49] "Lactate, ISTAT"                  "Lactate, Whole Bld"             
[51] "Lactic Acid"                     "Lymphocyte, Absolute"           
[53] "Lymphocytes, ABS (man diff)"     "MCH"                            
[55] "Monocyte, Absolute"              "Monocytes, ABS (man diff)"      
[57] "Neut, ABS (Seg+Band) (man diff)" "Neutrophil, Absolute"           
[59] "O2 Saturation (a)"               "O2 Saturation (v)"              
[61] "O2 Saturation, ISTAT"            "O2 Saturation, ISTAT (Ven)"     
[63] "Oxygen Saturation for POC"       "pCO2 (a)"                       
[65] "pCO2 (a), ISTAT"                 "pCO2 (v)"                       
[67] "PCO2 (v), ISTAT"                 "PCO2, ISTAT"                    
[69] "pH (a)"                          "PH (a), ISTAT"                  
[71] "pH (v)"                          "PH (v), ISTAT"                  
[73] "pH by Meter"                     "PH, ISTAT"                      
[75] "Platelet count"                  "pO2 (a)"                        
[77] "PO2 (a), ISTAT"                  "pO2 (v)"                        
[79] "PO2 (v), ISTAT"                  "PO2, ISTAT"                     
[81] "Potassium, ISTAT"                "Potassium, Ser/Plas"            
[83] "Potassium, Whole Bld"            "Potassium, whole blood, ePOC"   
[85] "Protein, Total, Ser/Plas"        "Prothrombin Time"               
[87] "PT, ISTAT"                       "RDW"                            
[89] "Sodium, ISTAT"                   "Sodium, Ser/Plas"               
[91] "Sodium, Whole Blood"             "Sodium, whole blood, ePOC"      
[93] "tCO2"                            "TCO2 (a), ISTAT"                
[95] "TCO2, ISTAT"                     "Total Bilirubin, Ser/Plas"      
[97] "TROPONIN I"                      "Troponin I, POCT"               
[99] "WBC"

In [12]:
# check each lab in the list for reference units, low and high to see if consistent among similar lab names
lab_test <- labs0 %>% select(lab_name, ord_num_value, reference_unit, reference_low, reference_high, base_name)# %>%
#                         mutate(reference_unit = factor(reference_unit),
#                                reference_low = factor(reference_low),
#                                reference_high = factor(reference_high)) #%>% 
#                 drop_na(ord_num_value)
c=0
for (l in lab_names){
    c = c+1
    lab <- lab_test %>% filter(lab_name == l & ord_num_value != 9999999)
    print(l)
    print(lab %>% group_by(base_name) %>% count())
    print(summary(lab %>% select(ord_num_value, reference_unit, reference_low, reference_high)))
}

# options(repr.plot.width=7, repr.plot.height=7)
print(c)

# 'Albumin, Ser/Plas', 'Alk P\'TASE, Total, Ser/Plas', 'ALT (SGPT), Ser/Plas', 'AST (SGOT), Ser/Plas'
# 'Base Excess (vt)', 'Base Excess Arterial for POC', 'Base Excess, ISTAT'
# 'Calcium, Ser/Plas''CO2, Ser/Plas''eGFR''Globulin''Glucose'
# 'Glucose Urine''Glucose, urine''Glucose, Urine'
# 'MCH''Neut, ABS (man diff)''Platelets''Potassium''Protein, Total, Ser/Plas'
# 'RDW''Sodium, whole blood, ePOC'

[1] "Albumin, Ser/Plas"
# A tibble: 1 x 2
# Groups:   base_name [1]
  base_name     n
  <chr>     <int>
1 ALB       14755
 ord_num_value   reference_unit  reference_low   reference_high 
 Min.   :1.100   g/dL   :14755   3.5    :14753   5.2    :12325  
 1st Qu.:3.400          :    0   3.2    :    2   5.0    : 2428  
 Median :3.900   %      :    0          :    0   4.5    :    2  
 Mean   :3.828   INR    :    0   >60    :    0          :    0  
 3rd Qu.:4.300   K/uL   :    0   >90    :    0   <0.055 :    0  
 Max.   :6.500   mEq/L  :    0   0.0    :    0   <0.1   :    0  
                 (Other):    0   (Other):    0   (Other):    0  
[1] "Alk P'TASE, Total, Ser/Plas"
# A tibble: 1 x 2
# Groups:   base_name [1]
  base_name     n
  <chr>     <int>
1 ALKP      14721
 ord_num_value    reference_unit  reference_low  reference_high
 Min.   :   9.0   U/L    :14721   40     :8906   130    :8906  
 1st Qu.:  69.0          :    0   35     :5757   105    :5757  
 Median :  90.0   %      :    0   

In [13]:
# update lab list, cut some labs - from TRIAGE
# OK to include 1 entry Glucose
# be careful with Lactate has 2 basename whole blood and none

# remove Potassium --> 1 entry, 7.4 weird
# remove Platelets --> all NA
# remove base_name of glucose urine --> all NA
# no longer here to remove: remove Basophils --> unit is %
# no longer here to remove: "pH" , "Ketone, urine", "Leukocyte Esterase, urine" # few, and all NA

lab_list <- c("RBC", "MCH", "MCHC", "MCV", "Calcium, Ser/Plas", "CO2, Ser/Plas",
               "Albumin, Ser/Plas", "ALT (SGPT), Ser/Plas", "AST (SGOT), Ser/Plas", "Alk P'TASE, Total, Ser/Plas",
               "Globulin", "Protein, Total, Ser/Plas",
               "Magnesium, Ser/Plas",  "Phosphorus, Ser/Plas", #14 ind
               
               "WBC", "WBC count", # WBC count is 1000*WBC
               "TROPONIN I", "Troponin I, POCT",
               "Total Bilirubin, Ser/Plas", "Total Bilirubin",
               "Platelet count", # remove Platelets
              
               "Hematocrit", "Hct, ISTAT", "Hct(Calc), ISTAT", "Hct (Est)", "HCT, POC", "Hematocrit (Manual Entry) See EMR for details",
               "Hemoglobin", "Hgb(Calc), ISTAT", "Hgb, calculated, POC", "HgB", 
               "Potassium, Ser/Plas", "Potassium, ISTAT",  "Potassium, Whole Bld", "Potassium, whole blood, ePOC", # remove Potassium
               "Sodium, Ser/Plas", "Sodium, ISTAT",  "Sodium, Whole Blood", 
               "Chloride, Ser/Plas",  "Chloride, ISTAT",  "Chloride, Whole Bld",
               "Creatinine, Ser/Plas", "Creatinine,ISTAT", 
              
               "Anion Gap", "Anion Gap, ISTAT",
               "Glucose, Ser/Plas", "Glucose,ISTAT", "Glucose, Whole Blood", "Glucose by Meter", "Glucose, Non-fasting",
               "BUN, Ser/Plas", "BUN, ISTAT", "Urea Nitrogen,Ser/Plas", 'BUN, Arterial', 'BUN, Peripheral', 'BUN, Venous',
               "Neutrophil, Absolute", "NEUT, ABS", "Neut, ABS (Seg+Band) (man diff)", "Neutrophils, Absolute (Manual Diff)", "Neut, ABS (Seg+Band) (man diff)",
               "Basophil, Absolute", "BASOS, ABS", "Basophils, ABS (man diff)", "Baso, ABS (man diff)", # remove Basophils
               "Eosinophil, Absolute", "EOS, ABS", "Eosinophils, ABS (man diff)", "Eos, ABS (man diff)",
               "Lymphocyte, Absolute", "LYM, ABS", "Lymphocytes, ABS (man diff)", "Lym, ABS (man diff)", "Lymphocytes, Abs.",
                  # Lymphocytes, Abs. = 1000* the rest of lymphocytes
               "Monocyte, Absolute", "MONO, ABS", "Monocytes, ABS (man diff)", "Mono, ABS (man diff)",
               "Lactate, ISTAT", "Lactate, Whole Bld", "Lactic Acid",
               "Base Excess, ISTAT", "Base Excess (vt)", "Base Excess Arterial for POC",
              
                "HCO3", "HCO3 (a), ISTAT", "Bicarbonate, Art for POC", 
                "HCO3 (v), ISTAT", "HCO3, ISTAT", 
                "O2 Saturation (a)", "O2 Saturation, ISTAT", "Oxygen Saturation for POC", 
                "O2 Saturation (v)",  "O2 Saturation, ISTAT (Ven)",
                "pCO2 (a)", "pCO2 (a), ISTAT",  "PCO2, ISTAT", "Arterial pCO2 for POC",
                "pCO2 (v)", "PCO2 (v), ISTAT",
                "pH (a)", "PH (a), ISTAT", "pH by Meter", "Arterial pH for POC",
                "pH (v)", "PH (v), ISTAT", "PH, ISTAT", 
                "pO2 (a)", "PO2 (a), ISTAT",  "PO2, ISTAT", "Arterial pO2 for POC",
                "pO2 (v)", "PO2 (v), ISTAT", 
              
                "tCO2", "TCO2 (a), ISTAT","TCO2, ISTAT", "TCO2, (ISTAT)", "CO2 Arterial Total for POC", 
                "TCO2 (v), ISTAT", # 1 individual
                "INR", "INR, ISTAT", 
                "Prothrombin Time", "PT, ISTAT") # 33 more than 1
#               "pH", "Ketone, urine", "Leukocyte Esterase, urine") #"O2 Saturation, ISTAT (Oth)", "ctO2 (a)"
length(lab_list)

[1] 123

In [14]:
colnames(labs0)

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "inpatient_data_id_coded" "admit_time"             
 [5] "label"                   "DBP"                    
 [7] "Pulse"                   "RR"                     
 [9] "SBP"                     "Temp"                   
[11] "order_id_coded"          "lab_name"               
[13] "base_name"               "ord_value"              
[15] "ord_num_value"           "reference_low"          
[17] "reference_high"          "reference_unit"         
[19] "result_in_range_yn"      "result_flag"            
[21] "result_time_utc"         "order_time_utc"         
[23] "taken_time_utc"

In [15]:
# sodium base_name is NA, so might not pick up. either collect is.na(base_name) or specify these lab_names
labs0 %>% filter(str_detect(lab_name, "Sodium")) %>% group_by(lab_name, base_name) %>% count() %>% distinct()

lab_name,base_name,n
<chr>,<chr>,<int>
"Sodium, ISTAT",NA,915
"Sodium, Ser/Plas",NA,15493
"Sodium, Whole Blood",NA,66
"Sodium, whole blood, ePOC",NA,32


In [16]:
# Only need to remove glucose in urine, 1 potassium, 1 platelets, and base_name of urine glucose
# changed labs0 to labs here
removed_lab_names <- c("Platelets", "Potassium")
labs <- labs0 %>% filter(!lab_name %in% removed_lab_names, base_name != "GLUURN" | is.na(base_name)) %>%
                select(-c(taken_time_utc, order_time_utc)) %>%
#             select(anon_id, pat_enc_csn_id_coded, lab_name, base_name, ord_num_value, 
#                    result_flag, order_time_utc, taken_time_utc, result_time_utc) %>%
                rename(features = lab_name, values = ord_num_value, result_time = result_time_utc) %>% 
#             mutate(feature_type = "labs", result_flag = ifelse(result_flag=="",0,1)) %>% 
                drop_na(values) %>% distinct()
nrow(labs)

[1] 513368

In [17]:
str_sort(setdiff(labs$lab_name, lab_list)) # NONE
str_sort(setdiff(lab_list, labs$features)) # ok
labs %>% filter(features %in% c('TCO2 (v), ISTAT')) # not there (only 1 person from the old dataset)
str_sort(unique(labs$base_name)) # has NA

character(0)

[1] "Anion Gap, ISTAT"                             
 [2] "Baso, ABS (man diff)"                         
 [3] "BASOS, ABS"                                   
 [4] "BUN, ISTAT"                                   
 [5] "Chloride, ISTAT"                              
 [6] "EOS, ABS"                                     
 [7] "Eos, ABS (man diff)"                          
 [8] "Hct(Calc), ISTAT"                             
 [9] "Hematocrit (Manual Entry) See EMR for details"
[10] "LYM, ABS"                                     
[11] "Lym, ABS (man diff)"                          
[12] "Lymphocytes, Abs."                            
[13] "Magnesium, Ser/Plas"                          
[14] "MCHC"                                         
[15] "MCV"                                          
[16] "MONO, ABS"                                    
[17] "Mono, ABS (man diff)"                         
[18] "NEUT, ABS"                                    
[19] "Neutrophils, Absolute (Manual Diff)"          
[20] "Phosphorus, Ser/Plas"                         
[21] "RBC"                                          
[22] "TCO2 (v), ISTAT"                              
[23] "TCO2, (ISTAT)"                                
[24] "Total Bilirubin"                              
[25] "Urea Nitrogen,Ser/Plas"                       
[26] "WBC count"

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label,DBP,Pulse,RR,SBP,Temp,⋯,features,base_name,ord_value,values,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,result_time
<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,⋯,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>


[1] "AG"      "ALB"     "ALKP"    "ALT"     "AST"     "BASOAB"  "BE"     
 [8] "BUN"     "CA"      "CL"      "CO2"     "CR"      "EGFR"    "EOSAB"  
[15] "GLOB"    "GLU"     "HCO3"    "HCO3A"   "HCO3V"   "HCT"     "HGB"    
[22] "INR"     "K"       "LAC"     "LACWBL"  "LYMAB"   "MCH"     "MONOAB" 
[29] "NEUTAB"  "NEUTABS" "O2SATA"  "O2SATV"  "PCO2A"   "PCO2V"   "PH"     
[36] "PHA"     "PHV"     "PLT"     "PO2A"    "PO2V"    "PT"      "RDW"    
[43] "TBIL"    "TCO2A"   "TNI"     "TP"      "WBC"     NA

In [18]:
# convert WBC count to WBC unit and Lymphocytes Abs. before combining
labs <- labs %>% mutate(values = ifelse(features == "WBC count", round(values/1000.0, 1), 
                                          ifelse(features == "Lymphocytes, Abs.", round(values/1000.0, 3), values))) 

In [19]:
# in range: Y or blank 
labs %>% count(result_in_range_yn) %>% arrange(desc(result_in_range_yn))
# normal as blanks
labs %>% count(result_flag) %>% arrange(desc(n))
head(labs %>% count(values) %>% arrange())
tail(labs %>% count(values) %>% arrange())

result_in_range_yn,n
<fct>,<int>
NA,513368


result_flag,n
<fct>,<int>
,347550
High,80665
Low,78550
High Panic,3595
Low Panic,3008


,values,n
,<dbl>,<int>
1,-31.2,1
2,-30.0,2
3,-29.0,1
4,-28.0,5
5,-27.0,7
6,-26.0,10


,values,n
,<dbl>,<int>
4815,3541,1
4816,4155,1
4817,4316,1
4818,4859,1
4819,5253,1
4820,9999999,6144


In [20]:
labs %>% group_by(features, base_name) %>% count() %>% distinct()

features,base_name,n
<chr>,<chr>,<int>
"Albumin, Ser/Plas",ALB,14755
"Alk P'TASE, Total, Ser/Plas",ALKP,14721
"ALT (SGPT), Ser/Plas",ALT,14710
Anion Gap,AG,15472
Arterial pCO2 for POC,PCO2A,1
Arterial pH for POC,PHA,1
Arterial pO2 for POC,PO2A,1
"AST (SGOT), Ser/Plas",AST,14702
Base Excess (vt),BE,333


In [21]:
###str_sort(unique(labs$features))

In [22]:


# ISTAT seems all A, so put pH in A
# combine same labs and change the lab name in the data
# could use base_name except for LACTATE has 2 different base_names
# Glucose lab_name as base_name as GLU and UGLU. exclude glucose here, only 1 entry anyways
# New: Glucose, Sodium ePOC, 'Neut, ABS (man diff)' and ALB - RDW

ALB = 'Albumin, Ser/Plas'
ALK = 'Alk P\'TASE, Total, Ser/Plas'
ALT = 'ALT (SGPT), Ser/Plas'
AST = 'AST (SGOT), Ser/Plas'

Ca = 'Calcium, Ser/Plas'
CO2 = 'CO2, Ser/Plas'
eGFR = 'eGFR' # as char
Glob = 'Globulin'

MCH = 'MCH' # as char
TProtein = 'Protein, Total, Ser/Plas'
RDW = 'RDW' # as char

Platelet = c("Platelet count") # remove platelets 
TBili = c("Total Bilirubin", "Total Bilirubin, Ser/Plas")
Trop = c("TROPONIN I", "Troponin I, POCT")
WBC = c("WBC", "WBC count") # WBC count /1000 7

Hct = c("Hct (Est)", "Hct, ISTAT", "HCT, POC", "Hct(Calc), ISTAT", "Hematocrit",    
        "Hematocrit (Manual Entry) See EMR for details")
Hgb = c("Hemoglobin", "HgB", "Hgb, calculated, POC", "Hgb(Calc), ISTAT")
K = c("Potassium, ISTAT",  "Potassium, Ser/Plas", "Potassium, Whole Bld", "Potassium, whole blood, ePOC") # remove Potassium
Na = c("Sodium, ISTAT",  "Sodium, Ser/Plas", "Sodium, Whole Blood", 'Sodium, whole blood, ePOC') # ePOC is new
Cl = c("Chloride, ISTAT",  "Chloride, Ser/Plas",  "Chloride, Whole Bld")
Cr = c("Creatinine, Ser/Plas", "Creatinine,ISTAT") #22

AnionGap = c("Anion Gap", "Anion Gap, ISTAT") 
Glucose = c('Glucose', "Glucose by Meter", "Glucose, Non-fasting", "Glucose, Ser/Plas", "Glucose, Whole Blood", "Glucose,ISTAT")
BUN = c("BUN, Ser/Plas", "BUN, ISTAT", "Urea Nitrogen,Ser/Plas", 'BUN, Arterial', 'BUN, Peripheral', 'BUN, Venous') #10
Neut = c("NEUT, ABS", 'Neut, ABS (man diff)', "Neut, ABS (Seg+Band) (man diff)", "Neutrophil, Absolute",  "Neutrophils, Absolute (Manual Diff)")
Basos = c("Basophils, ABS (man diff)", "Basophil, Absolute", "Baso, ABS (man diff)", "BASOS, ABS")
Eos = c("EOS, ABS", "Eosinophils, ABS (man diff)", "Eosinophil, Absolute", "Eos, ABS (man diff)")
Lymp = c("LYM, ABS", "Lymphocytes, ABS (man diff)", "Lymphocyte, Absolute", "Lym, ABS (man diff)", "Lymphocytes, Abs.")
Mono = c("MONO, ABS", "Monocytes, ABS (man diff)", "Monocyte, Absolute", "Mono, ABS (man diff)")
Lactate = c("Lactate, ISTAT", "Lactate, Whole Bld", "Lactic Acid")
Base = c("Base Excess (vt)", "Base Excess Arterial for POC", "Base Excess, ISTAT") #28

HCO3_a = c("HCO3", "HCO3 (a), ISTAT", "Bicarbonate, Art for POC") # HCO3 is A based on base name
HCO3_v = c("HCO3 (v), ISTAT", "HCO3, ISTAT") 
O2sat_a = c("O2 Saturation (a)", "O2 Saturation, ISTAT", "Oxygen Saturation for POC")
O2sat_v = c("O2 Saturation (v)",  "O2 Saturation, ISTAT (Ven)")
pCO2_a = c("pCO2 (a)", "pCO2 (a), ISTAT",  "PCO2, ISTAT", "Arterial pCO2 for POC")
pCO2_v = c("pCO2 (v)", "PCO2 (v), ISTAT") 
pH_a = c("pH (a)", "PH (a), ISTAT", "pH by Meter", "Arterial pH for POC", "PH, ISTAT")
pH_v = c ("pH (v)", "PH (v), ISTAT") # new: move "PH, ISTAT" to arterial
PO2_a = c("pO2 (a)", "PO2 (a), ISTAT",  "PO2, ISTAT", "Arterial pO2 for POC")
PO2_v = c ("pO2 (v)", "PO2 (v), ISTAT") #29

TCO2_a = c("tCO2", "TCO2 (a), ISTAT", "TCO2, (ISTAT)", "TCO2, ISTAT", "CO2 Arterial Total for POC")
INR = c("INR", "INR, ISTAT")
PT = c("Prothrombin Time", "PT, ISTAT") #9

labs <- labs %>% 
        mutate(features = 
            ifelse(features %in% Platelet, "Platelet", ifelse(features %in% TBili, "TBili", 
            ifelse(features %in% Trop, "Trop", ifelse(features %in% WBC, "WBC",
            ifelse(features %in% Hct, "Hct", ifelse(features %in% Hgb, "Hgb", 
            ifelse(features %in% K, "K", ifelse(features %in% Na, "Na",
            ifelse(features %in% Cl, "Cl", ifelse(features %in% Cr, "Cr",
            ifelse(features %in% AnionGap, "AnionGap", ifelse(features %in% Glucose, "Glucose",
            ifelse(features %in% BUN, "BUN", ifelse(features %in% Neut, "Neut",    
            ifelse(features %in% Basos, "Basos", ifelse(features %in% Eos, "Eos",      
            ifelse(features %in% Lymp, "Lymp", ifelse(features %in% Mono, "Mono",    
            ifelse(features %in% Lactate, "Lactate",  ifelse(features %in% Base, "Base", 
            ifelse(features %in% HCO3_a, "HCO3_a", ifelse(features %in% HCO3_v, "HCO3_v", 
            ifelse(features %in% O2sat_a, "O2sat_a", ifelse(features %in% O2sat_v, "O2sat_v",
            ifelse(features %in% pCO2_a, "pCO2_a", ifelse(features %in% pCO2_v, "pCO2_v",
            ifelse(features %in% pH_a, "pH_a", ifelse(features %in% pH_v, "pH_v",      
            ifelse(features %in% PO2_a, "PO2_a", ifelse(features %in% PO2_v, "PO2_v",
            ifelse(features %in% TCO2_a, "TCO2_a", ifelse(features %in% INR, "INR",
            ifelse(features %in% PT, "PT", ifelse(features %in% ALB, "ALB",
            ifelse(features %in% ALK, "ALK", ifelse(features %in% ALT, "ALT",
            ifelse(features %in% AST, "AST", ifelse(features %in% TProtein, "TProtein",                                     
            ifelse(features %in% Ca, "Ca", ifelse(features %in% CO2, "CO2",                                        
            ifelse(features %in% Glob, "Glob", as.character(features))))))))))))))))))))))))))))))))))))))))))) %>%
        distinct() 


In [23]:
nrow(labs) # 1367892 vs 1367838
# total 48 lab_name but 56 base_name
nrow(labs %>% select(anon_id, pat_enc_csn_id_coded) %>% distinct()) #41627
# labs %>% select(base_name, features) %>% distinct() %>% arrange(features, base_name) 
labs %>% group_by(features, base_name) %>% count(sort=TRUE) %>% arrange(features, base_name) 

[1] 513368

[1] 15982

features,base_name,n
<chr>,<chr>,<int>
ALB,ALB,14755
ALK,ALKP,14721
ALT,ALT,14710
AnionGap,AG,15472
AST,AST,14702
Base,BE,8018
Basos,BASOAB,13775
BUN,BUN,15480
Ca,CA,15501


### Process the 9999999 values
- check ord_value

In [24]:
# there are no NA, but only 15668 of 9999999 values
value99 <- labs %>% filter(values==9999999) 
nrow(value99)
value99 %>% group_by(ord_value) %>% count() %>% arrange(-n)

[1] 6144

ord_value,n
<chr>,<int>
<0.017,4442
>90,811
<0.2,446
<0.30,91
<5,75
>500,69
>600,32
>700,20
<15,19


In [25]:
value99 %>% filter(ord_value %in% c("N/A", "hi") | str_detect(ord_value, 'CANCELLED')) %>% count(ord_value, sort=TRUE)
value99 %>% filter(!grepl('>|<', ord_value)) %>% count(ord_value, sort=TRUE)

value99 %>% filter(ord_value %in% c("N/A", "hi") | str_detect(ord_value, 'CANCELLED')) %>% count(features, sort=TRUE)

ord_value,n
<chr>,<int>


ord_value,n
<chr>,<int>


features,n
<chr>,<int>


In [26]:
# number of cancelled, NA, or 1 hi observations under ord_value
nrow(labs %>% filter(values==9999999, !grepl('>|<', ord_value)) %>% mutate(ord_value = factor(ord_value))) # %>% group_by(ord_value) %>% count() %>% arrange(-n)

[1] 0

In [27]:
# revmove those without > or < in ord_value (416), work with this set and then do it on the main labs
# take the upper/lower bounds for the 9999999 values and add 1% to it
value99 <- value99 %>% filter(grepl('>|<', ord_value)) %>% #mutate(ord_value = as.character(ord_value)) %>%
            mutate(values = 1.01 * as.double(gsub(paste(c(">", "<"), collapse = "|"),"", ord_value))) #%>%
#             drop_na(values) # %>% distinct() # same

nrow(value99) #15250
summary(value99$values)

[1] 6144

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
 -30.300    0.017    0.017   25.340    0.202 7070.000 

In [28]:
head(value99)

,anon_id,pat_enc_csn_id_coded,inpatient_data_id_coded,admit_time,label,DBP,Pulse,RR,SBP,Temp,⋯,features,base_name,ord_value,values,reference_low,reference_high,reference_unit,result_in_range_yn,result_flag,result_time
,<chr>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<dbl>,⋯,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>
1,JC2948274,131288452994,53606177,2020-06-05 00:58:00+00:00,1,59,80,18,114,36.15,⋯,Lactate,LACWBL,>15.0,15.15,,<2.0,mmol/L,NA,High Panic,2020-06-05 00:15:00+00:00
2,JC574617,131302008119,61925940,2021-02-15 07:47:00+00:00,1,42,78,20,91,35.05,⋯,Lactate,LACWBL,>15.0,15.15,,<2.0,mmol/L,NA,High Panic,2021-02-15 06:11:00+00:00
3,JC3035557,131298105782,59568658,2020-10-30 21:00:00+00:00,0,89,89,17,140,37.00,⋯,pCO2_a,PCO2A,>110.0,111.10,35.0,45.0,mm Hg,NA,High Panic,2020-10-30 20:31:00+00:00
4,JC1263713,131290915323,55153388,2020-08-04 03:40:00+00:00,1,106,82,29,161,35.00,⋯,TCO2_a,TCO2A,<5,5.05,23,27,mmol/L,NA,Low Panic,2020-08-04 03:25:00+00:00
5,JC574617,131302008119,61925940,2021-02-15 07:47:00+00:00,1,42,78,20,91,35.05,⋯,Lactate,LACWBL,>15.0,15.15,,<2.0,mmol/L,NA,High Panic,2021-02-15 07:31:00+00:00
6,JC3035557,131298105782,59568658,2020-10-30 21:00:00+00:00,0,89,89,17,140,37.00,⋯,PO2_a,PO2A,<30.0,30.30,80.0,105.0,mm Hg,NA,Low Panic,2020-10-30 20:31:00+00:00


In [29]:
# remove only the observations (where values = 999999 and do not have < or > in ord_value) from process labs
# could do 1.01 or 0.99 but more complicated
grepl('>|<', ">60")
labs99 <- labs %>% filter(!(values==9999999 & !grepl('>|<', ord_value))) %>%
            mutate(values = as.double(gsub(paste(c(">", "<"), collapse = "|"),"", ord_value)),
                   feature_type = "labs") 
nrow(labs) - nrow(labs99) # 1367892 - 1367476
nrow(labs99) # 1367476 vs 1367422
summary(labs99$values)
colnames(labs99)

[1] TRUE

[1] 0

[1] 513368

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -31.20    3.20   12.30   39.01   38.00 7000.00 

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "inpatient_data_id_coded" "admit_time"             
 [5] "label"                   "DBP"                    
 [7] "Pulse"                   "RR"                     
 [9] "SBP"                     "Temp"                   
[11] "order_id_coded"          "features"               
[13] "base_name"               "ord_value"              
[15] "values"                  "reference_low"          
[17] "reference_high"          "reference_unit"         
[19] "result_in_range_yn"      "result_flag"            
[21] "result_time"             "feature_type"

In [30]:
labs99 <- labs99 %>% select(-c('DBP','Pulse','RR','SBP','Temp', 'order_id_coded'))
colnames(labs99)

[1] "anon_id"                 "pat_enc_csn_id_coded"   
 [3] "inpatient_data_id_coded" "admit_time"             
 [5] "label"                   "features"               
 [7] "base_name"               "ord_value"              
 [9] "values"                  "reference_low"          
[11] "reference_high"          "reference_unit"         
[13] "result_in_range_yn"      "result_flag"            
[15] "result_time"             "feature_type"

In [31]:
write.csv(labs99, file.path(valdir, "6_6_coh3_labs.csv"), row.names=FALSE)

### Keep this cohort information:

In [32]:
cohort_with_labs <- labs99 %>% select(anon_id, pat_enc_csn_id_coded, inpatient_data_id_coded, admit_time, label) %>% distinct()
nrow(cohort_with_labs)

cohort_with_labs %>% group_by(label) %>% summarise(count = n(), percent = round(100*count/nrow(cohort_with_labs),2))

[1] 15982

label,count,percent
<int>,<int>,<dbl>
0,13760,86.1
1,2222,13.9


In [33]:
write.csv(cohort_with_labs, file.path(valdir, "6_6_cohort3_withlabs.csv"), row.names=FALSE)